In [1]:
import pandas as pd
import sqlite3 
from faker import Faker
import random
from datetime import datetime

In [2]:
df_channels = pd.read_excel('Channels_db.xlsx', index_col=0)
df_videos = pd.read_excel('Videos_db.xlsx', index_col=0)
df_comments = pd.read_excel('Comments_db.xlsx', index_col=0)
df_categories = pd.read_excel('Categories_db.xlsx', index_col=0)
df_projects = pd.read_excel('Projects_db.xlsx', index_col=0)
df_videos_cats = pd.read_excel('Videos_categories_db.xlsx', index_col=0)

df_dict = {
    "Channels": df_channels,
    "Videos": df_videos,
    "Comments": df_comments,
    "Categories": df_categories,
    "Projects": df_projects,
    "Videos_Categories": df_videos_cats
}

for name, df in df_dict.items():
    print(f"DataFrame: {name}")
    display(df.head())
    print('-----')

DataFrame: Channels


,ID,CreationDate,Title,Description,SubscriberCount
1,UCnDgIiMU3CyNHGP9wlLbgqw,2019-06-21,LABELSMART,Образовательно-развлекательные разговорные фор...,1180000
2,UCXDerjAwOSH30mrOW80hQjA,2020-09-09,easycom,Easycom - канал с комиками. Новые форматы. Нов...,1180000
3,UCNqktdxgAADBj36dC7VGOgg,2016-08-30,LABELCOM,Это канал LABELCOM \n\nПо вопросам рекламы и с...,7350000


-----
DataFrame: Videos


,ID,Channel_ID,Project_ID,Title,Description,CreationDate,Duration,LikeCount,CommentCount,ViewCount,DislikeCount
1,KyAT9ERav2g,UCnDgIiMU3CyNHGP9wlLbgqw,5,"ПРО КОМАНДУ КВН ""ХЛОБЫСЬ"" #мнесмешно #моргунов...",NaN,2024-11-24,56,2305,20,87631,50
2,Ink77itokMI,UCnDgIiMU3CyNHGP9wlLbgqw,9,АНЕКДОТ ОТ КАТИ МОРГУНОВОЙ #мнесмешно #моргуно...,NaN,2024-11-13,54,26620,492,1077597,1880
3,-XByR0UZUJg,UCnDgIiMU3CyNHGP9wlLbgqw,4,П🫣РНОСТАР #аргументы,Смотри выпуск только в VK видео https://vk.cc/...,2023-07-10,16,107085,556,4621875,3516
4,OoVg3-zOOCw,UCnDgIiMU3CyNHGP9wlLbgqw,1,КТО В СЕМЬЕ ОТВЕЧАЕТ ЗА БЮДЖЕТ? #lockstock #me...,NaN,2024-06-30,15,105198,240,3013391,5221
5,9XLv5F3mBVY,UCnDgIiMU3CyNHGP9wlLbgqw,9,ОЧЕНЬ ЖЕСТКИЙ АНЕКДОТ #мнесмешно,NaN,2023-12-14,49,823,22,43273,19


-----
DataFrame: Comments


...

-----
DataFrame: Projects


,ID,Title,Description
0,1,Мир Смеха,Проект о юморе и комедии.
1,2,Горячие минуты,Проект о банях и отдыхе.
2,3,Темные дела,Проект о криминале и расследованиях.
3,4,Аргументы и факты,Проект о фактах и аргументах.
4,5,Ночная история,Проект об историях на ночь.


-----
DataFrame: Videos_Categories


,Video_ID,Category_ID
0,KyAT9ERav2g,5
1,Ink77itokMI,5
2,-XByR0UZUJg,5
3,OoVg3-zOOCw,5
4,9XLv5F3mBVY,5


-----


In [51]:
# Создание базы

connection =  sqlite3.connect("yt_project.db")
cursor = connection.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS Videos (
    ID TEXT PRIMARY KEY NOT NULL,
    Channel_ID INTEGER NOT NULL,
    Project_ID INTEGER,
    Title TEXT NOT NULL,
    Description TEXT,
    CreationDate DATE,
    Duration INTEGER CHECK (Duration >= 0),
    LikeCount INTEGER CHECK (LikeCount >= 0),
    DislikeCount INTEGER CHECK (DislikeCount >= 0),
    CommentCount INTEGER CHECK (CommentCount >= 0),
    ViewCount INTEGER CHECK (ViewCount >= 0),
    FOREIGN KEY (Channel_ID) REFERENCES Channels (ID) ON DELETE CASCADE
    FOREIGN KEY (Project_ID) REFERENCES Projects (ID) ON DELETE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Channels (
    ID TEXT PRIMARY KEY NOT NULL,
    Title TEXT NOT NULL,
    Description TEXT,
    CreationDate DATE,
    SubscriberCount INTEGER CHECK (SubscriberCount >= 0)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Comments (
    ID TEXT PRIMARY KEY NOT NULL,
    Video_ID INTEGER NOT NULL,
    Text TEXT NOT NULL,
    CreationDate DATE,
    LikeCount INTEGER CHECK (LikeCount >= 0),
    Sentiment INTEGER,
    FOREIGN KEY (Video_ID) REFERENCES Videos (ID)
        ON DELETE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Videos_Categories (
    Video_ID INTEGER NOT NULL,
    Category_ID INTEGER NOT NULL,
    PRIMARY KEY (Video_ID, Category_ID),
    FOREIGN KEY (Video_ID) REFERENCES Videos (ID) ON DELETE CASCADE
    FOREIGN KEY (Category_ID) REFERENCES Categories (ID) ON DELETE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Categories (
    ID INTEGER PRIMARY KEY NOT NULL,
    Title TEXT NOT NULL
)
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Projects (
    ID INTEGER PRIMARY KEY NOT NULL,
    Title TEXT NOT NULL,
    Description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Costs (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Project_ID INTEGER NOT NULL,
    Category TEXT NOT NULL,
    Counterparty TEXT NOT NULL,
    CreationDate DATE,
    Amount INTEGER CHECK (Amount >= 0),
    FOREIGN KEY (Project_ID) REFERENCES Projects (ID) ON DELETE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Incomes (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Project_ID INTEGER NOT NULL,
    Category TEXT NOT NULL,
    Counterparty TEXT NOT NULL,
    CreationDate DATE,
    Amount INTEGER CHECK (Amount >= 0),
    FOREIGN KEY (Project_ID) REFERENCES Projects (ID) ON DELETE CASCADE
)
''')

connection.commit()
connection.close()

In [52]:
# Генерация таблиц incomes и costs

expenses_categories = [
    "Производство видео", "Монтаж", "Оборудование", "Аренда студии",
    "Гонорары актёрам", "Маркетинг и реклама", "Пропускная способность и хостинг",
    "Графический дизайн", "Звукозапись и микширование", "Путевые расходы",
    "Костюмы и реквизит", "Авторские права и лицензии", "Обучение и развитие персонала",
    "Административные расходы", "Техническая поддержка", "Консультации и юридические услуги",
    "Мероприятия и встречи", "Подписки на программное обеспечение",
    "Поддержка социальных сетей", "Аналитика и исследования"]

income_categories = [
    "Реклама на YouTube", "Спонсорство", "Продажа мерча", "Платные подписки",
    "Доход от партнерских программ", "Донаты и пожертвования", "Проведение мероприятий",
    "Онлайн-курсы и мастер-классы", "Продажа билетов на шоу", "Партнёрские ссылки и программы",
    "Лицензирование контента", "Платные вебинары", "Продажа цифрового контента",
    "Услуги по созданию контента", "Консультационные услуги", "Финансирование краудфандинга",
    "Роялти и лицензионные отчисления", "Реклама в социальных сетях",
    "Продажа цифровых продуктов", "Премиум-контент"]

random.seed(10)

def generate_financial_data(num_records, data_type='costs', min_amount=50000, max_amount=900000):
    data = []
    categories = expenses_categories if data_type == 'costs' else income_categories
    
    for _ in range(num_records):
        project_id = random.choice(df_projects['ID'].to_list())
        category = random.choice(categories)
        counterparty = fake.company()
        creation_date = fake.date_between(start_date=start_date, end_date=end_date)
        amount = random.randint(min_amount, max_amount)
        data.append((project_id, category, counterparty, creation_date, amount))
    
    return data

connection = sqlite3.connect("yt_project.db")
cursor = connection.cursor()

fake = Faker('ru_RU')

start_date = datetime(2016, 8, 30)
end_date = datetime(2024, 12, 23)

costs_data = generate_financial_data(30)
incomes_data = generate_financial_data(30, data_type='incomes')

cursor.executemany(''' INSERT INTO Costs (Project_ID, Category, Counterparty, CreationDate, Amount)
                       VALUES (?, ?, ?, ?, ?) ''', costs_data)

cursor.executemany(''' INSERT INTO Incomes (Project_ID, Category, Counterparty, CreationDate, Amount)
                       VALUES (?, ?, ?, ?, ?) ''', incomes_data)

connection.commit()
connection.close()


In [54]:
# Добавление остальных датафреймов

connection = sqlite3.connect("yt_project.db")

for table_name, df in df_dict.items():
    df.to_sql(table_name, connection, if_exists='append', index=False)

connection.commit()
connection.close()